# 3Satisfactory problem

3-CNF with one exact solution has $2^n - 1$ clauses
Like, 3 bits having 8 possible values. Removing one and the solution is that.

In [1]:
n = 3
remove = "001"
xyz = [0, 0, 1]

for i in range(2**n):
    clause = f'{i:03b}'
    if remove != clause:
        # if literal is 0 do nothing, if 1 invert the value
        value = [xyz[i] ^ int(c) for i, c in enumerate(clause)]
        print(value, any(value))

[0, 0, 1] True
[0, 1, 1] True
[0, 1, 0] True
[1, 0, 1] True
[1, 0, 0] True
[1, 1, 1] True
[1, 1, 0] True


In [2]:
def EvaluateThreeSat(remove: list, state: list):
    remove = ''.join(map(str, remove))
    for i in range(2**len(remove)):
        clause = format(i, f'0{len(remove)}b')
        if remove != clause:
            # if literal is 0 do nothing, if 1 invert the value
            value = [int(x) ^ int(c) for x, c in zip(state, clause)]
            if not any(value):
                return False
    return True

print(EvaluateThreeSat([0, 1, 1, 1], [0, 1, 1, 1]))

True


## Bruteforce

In [3]:
import numpy as np

randomized_problem = np.random.randint(0, 2, 10)
n = len(randomized_problem)

for i in range(2**n):
    evaluate = EvaluateThreeSat(randomized_problem, [int(c) for c in f'{i:010b}'])
    print(f"brute: {i:010b} {evaluate}")
    if evaluate:
        break    

brute: 0000000000 False
brute: 0000000001 False
brute: 0000000010 False
brute: 0000000011 False
brute: 0000000100 False
brute: 0000000101 False
brute: 0000000110 False
brute: 0000000111 False
brute: 0000001000 False
brute: 0000001001 False
brute: 0000001010 False
brute: 0000001011 False
brute: 0000001100 False
brute: 0000001101 False
brute: 0000001110 False
brute: 0000001111 False
brute: 0000010000 False
brute: 0000010001 False
brute: 0000010010 False
brute: 0000010011 False
brute: 0000010100 False
brute: 0000010101 False
brute: 0000010110 False
brute: 0000010111 False
brute: 0000011000 False
brute: 0000011001 False
brute: 0000011010 False
brute: 0000011011 False
brute: 0000011100 False
brute: 0000011101 False
brute: 0000011110 False
brute: 0000011111 False
brute: 0000100000 False
brute: 0000100001 False
brute: 0000100010 False
brute: 0000100011 False
brute: 0000100100 False
brute: 0000100101 False
brute: 0000100110 False
brute: 0000100111 False
brute: 0000101000 False
brute: 000010100

## QUBO Choi approach
MIS reduction\
n variables and m clauses and an empty graph $G_{SAT} = (V_{G_{SAT}}, E_{G_{SAT}})$\
For each claus $C_i$ = $l_{i, 1}$ v $l_{i, 2}$ v $l_{i, 3}$ we add a new 3 node clique to $G_{sat}$. The three nodes are labeled by the literals $l_{i, 1}$ , $l_{i, 2}$ and $l_{i, 3}$ of clause $C_i$\
A connection between nodes of differnet 3-node cliques will be added, if the labels of these nodes are in conflict. That is, iff $l_{i, s}$ = $\neg l_{j, t}$ for clause indices i $\not =$ j and s, t $\in$ {1, 2, 3} we add edge ($l_{i,s}$, $l_{j, t}$) to $G_{SAT}$

Given f(x) = (not x2 v not x3 x4) ^ (x1 v x2 v x4) ^ (not x1 v x3 v not x4)\
x1 = 1 x2 = 0 x3 = 1 satisfies f(x)

https://web.mit.edu/~neboat/www/6.046-fa09/rec8.pdf

In [4]:
from qiskit_algorithms import SamplingVQE
from qiskit_algorithms.optimizers import SPSA
from qiskit_algorithms.utils import algorithm_globals
from qiskit.primitives import Sampler
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization import QuadraticProgram


(a b c) (a !b d) 

In [5]:
# Initialize the quadratic program
qp = QuadraticProgram()

# Add binary variables x1, x2, x3, and the ancilla variable a_x1_x2
qp.binary_var('x1')
qp.binary_var('x2')
qp.binary_var('x3')
qp.binary_var('x4')
qp.binary_var('a1')
qp.binary_var('a2')

var = ['x1', 'x2', 'x3', 'x4', 'a1', 'a2']

Q = [[-3, 1, 1, 1, 1, 1],
     [0, -2, 1, 0, 1, 1],
     [0, 0, -2, 0, 1, 0],
     [0, 0, 0, -1, 0, 1],
     [0, 0, 0, 0, -2, 0],
     [0, 0, 0, 0, 0, -1]]

qp.minimize(quadratic= Q)
# Print the problem
print(qp.prettyprint())

Problem name: 

Minimize
  -2*a1^2 - a2^2 + x1*a1 + x1*a2 - 3*x1^2 + x1*x2 + x1*x3 + x1*x4 + x2*a1
  + x2*a2 - 2*x2^2 + x2*x3 + x3*a1 - 2*x3^2 + x4*a2 - x4^2

Subject to
  No constraints

  Binary variables (6)
    x1 x2 x3 x4 a1 a2



In [6]:
from qiskit_optimization.converters import QuadraticProgramToQubo

qp2qubo = QuadraticProgramToQubo()
qubo = qp2qubo.convert(qp)
qubitOp, offset = qubo.to_ising()

print('Offset:', offset)
print('Ising Hamiltonian:')
print(str(qubitOp))

Offset: -3.0
Ising Hamiltonian:
SparsePauliOp(['IIIIIZ', 'IIIIZZ', 'IIIZIZ', 'IIIZII', 'IIZIIZ', 'IZIIIZ', 'IZIIII', 'ZIIIIZ', 'ZIIIII', 'IIIZZI', 'IZIIZI', 'ZIIIZI', 'IZIZII', 'ZIZIII'],
              coeffs=[ 0.25+0.j,  0.25+0.j,  0.25+0.j,  0.25+0.j,  0.25+0.j,  0.25+0.j,
  0.25+0.j,  0.25+0.j, -0.25+0.j,  0.25+0.j,  0.25+0.j,  0.25+0.j,
  0.25+0.j,  0.25+0.j])


In [7]:
exact = MinimumEigenOptimizer(NumPyMinimumEigensolver())
result = exact.solve(qp)
print(result.prettyprint())

objective function value: -4.0
variable values: x1=1.0, x2=0.0, x3=1.0, x4=1.0, a1=1.0, a2=0.0
status: SUCCESS


$C(\textbf{x}) = \sum_{k = 1}^m c_k(\textbf{x})$

https://wigner.hu/~koniorczykmatyas/qubo/literature/1811.11538.pdf

https://hal.science/hal-04172858/document
Maximum independent set\
(a v b v c) (!a v b v c) (a v !b v c) (!a v b v !c)

In [8]:
import numpy as np
QUBO_matrix = np.array([[0, -2, 2, -1, 1, -2, 1],
                        [0, 1, 0, -1, -1, 1, -1],
                        [0, 0, -1, -1, -1, -1, 1],
                        [0, 0, 0, 2, 0, 0, 0],
                        [0, 0, 0, 0, 1, 0, 0],
                        [0, 0, 0, 0, 0, 1, 0],
                        [0, 0, 0, 0, 0, 0, 0]])

qp = QuadraticProgram()
for i in range(7):
    qp.binary_var(f'x{i}')

qp.minimize(quadratic=QUBO_matrix)
print(qp.prettyprint())
qp2qubo = QuadraticProgramToQubo()
qubo = qp2qubo.convert(qp)
qubitOp, offset = qubo.to_ising()


Problem name: 

Minimize
  -2*x0*x1 + 2*x0*x2 - x0*x3 + x0*x4 - 2*x0*x5 + x0*x6 + x1^2 - x1*x3 - x1*x4
  + x1*x5 - x1*x6 - x2^2 - x2*x3 - x2*x4 - x2*x5 + x2*x6 + 2*x3^2 + x4^2 + x5^2

Subject to
  No constraints

  Binary variables (7)
    x0 x1 x2 x3 x4 x5 x6



In [9]:
from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_optimization.algorithms import MinimumEigenOptimizer


exact = MinimumEigenOptimizer(NumPyMinimumEigensolver())

result = exact.solve(qubo)
print(result.prettyprint())

objective function value: -2.0
variable values: x0=1.0, x1=1.0, x2=1.0, x3=1.0, x4=1.0, x5=1.0, x6=0.0
status: SUCCESS


(a v b v c) (a v !b v d)

In [10]:
QUBO_matrix = np.array([[-1, 0, 1, 1, -1, -1],
                        [0, 0, 1, -1, -1, 1],
                        [0, 0, -1, 0, -1, 0],
                        [0, 0, 0, 0, 0, -1],
                        [0, 0, 0, 0, 2, 0],
                        [0, 0, 0, 0, 0, 1]])

qp = QuadraticProgram()
for i in range(6):
    qp.binary_var(f'x{i}')

qp.minimize(quadratic=QUBO_matrix)
print(qp.prettyprint())
qp2qubo = QuadraticProgramToQubo()
qubo = qp2qubo.convert(qp)
qubitOp, offset = qubo.to_ising()


exact = MinimumEigenOptimizer(NumPyMinimumEigensolver())

result = exact.solve(qubo)
print(result.prettyprint())

Problem name: 

Minimize
  -x0^2 + x0*x2 + x0*x3 - x0*x4 - x0*x5 + x1*x2 - x1*x3 - x1*x4 + x1*x5 - x2^2
  - x2*x4 - x3*x5 + 2*x4^2 + x5^2

Subject to
  No constraints

  Binary variables (6)
    x0 x1 x2 x3 x4 x5

objective function value: -1.0
variable values: x0=1.0, x1=1.0, x2=1.0, x3=1.0, x4=1.0, x5=0.0
status: SUCCESS
